# Reproduce LineVul

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import commons

In [3]:
from transformers import AutoTokenizer, AutoModel
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig

In [4]:
# there are some warning from transformer
# due to its verbose, disable

from transformers import logging
logging.set_verbosity(40)

In [5]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler

In [6]:
import numpy as np
from datasets import Dataset
from torch.utils.data import DataLoader

In [7]:
import pandas as pd
from tqdm.autonotebook import tqdm

In [8]:
from linevul_model import Model
from linevul_helpers import TextDataset
from linevul_extra import extract_line_attention, linevul_predict

In [9]:
from project_dataset import load_dataset

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [11]:
config = RobertaConfig.from_pretrained('microsoft/codebert-base')
config.num_labels = 1
config.num_attention_heads = 12

In [12]:
# get from LineVul
checkpoint = '/home/hqn650/LineVul/linevul/saved_models/checkpoint-best-f1/12heads_linevul_model.bin'

In [13]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

In [17]:
pre_train = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', 
                                                             config=config, 
                                                             ignore_mismatched_sizes=True).to(device)

In [18]:
from dataclasses import dataclass

@dataclass
class Args:
    device = device
    n_gpu = n_gpu
    use_non_pretrained_model = False
    block_size = 512
    test_data_file = '/home/hqn650/LineVul/data/big-vul_dataset/test.csv'
    code_length=256
    do_local_explanation=True
    reasoning_method='attention'
    seed=42
    num_attention_heads=12
    do_sorting_by_line_scores=False
    do_sorting_by_pred_prob=False
    top_k_constant=10
    use_word_level_tokenizer=False
    eval_batch_size=512

    task = "impact"
    
args = Args()

In [19]:
model = Model(pre_train, config, tokenizer, args)

In [20]:
model.load_state_dict(torch.load(checkpoint, map_location=args.device))
model.to(args.device)

Model(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((76

In [21]:
from linevul_helpers import TextDataset, convert_examples_to_features

class ExtendTextDataset(TextDataset):
    def __init__(self, tokenizer, args, data_frame):
        self.examples = []
        funcs = data_frame["processed_func"].tolist()
        for i in tqdm(range(len(funcs)), desc='ExtendTextDataset'):
            self.examples.append(convert_examples_to_features(funcs[i], 1, tokenizer, args))

In [22]:
# multi-gpu evaluate
if args.n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [23]:
# to find TP
def find_tp(model, tokenizer, args, data_frame=None):
    if data_frame is not None:
        dataset = ExtendTextDataset(tokenizer, args, data_frame)
    else:
        dataset = TextDataset(tokenizer, args, file_type='test')
    sampler = SequentialSampler(dataset)
    data_loader = DataLoader(dataset, sampler=sampler, batch_size=args.eval_batch_size, num_workers=0)
    result, y_trues, y_preds = linevul_predict(model, data_loader, args.device)
    tp_indices = np.where((y_trues == y_preds) & (y_trues == 1))
    tp_indices = list(tp_indices[0])
    return result, tp_indices
    
# result, correct_indices = find_tp(model, tokenizer, args)

In [24]:
def explain(model, tokenizer, explain_indices, data_frame=None): 
    """ 
        return (sample_idx, lines, n_lines)
    """
    if data_frame is not None:
        dataset = ExtendTextDataset(tokenizer, args, data_frame)
    else:
        dataset = TextDataset(tokenizer, args, file_type='test')
    sampler = SequentialSampler(dataset)
    data_loader = DataLoader(dataset, sampler=sampler, batch_size=1, num_workers=0)
    model.eval()
    index = 0
    progress_bar = tqdm(data_loader, total=len(data_loader))
    extract_list = []
    for mini_batch in progress_bar:
        if index in explain_indices:
            (input_ids, labels) = mini_batch
            ids = input_ids[0].detach().tolist()
            all_tokens = tokenizer.convert_ids_to_tokens(ids)
            all_tokens = [token.replace("Ġ", "") for token in all_tokens]
            all_tokens = [token.replace("ĉ", "Ċ") for token in all_tokens]
            with torch.no_grad():
                prob, attentions = model(input_ids=input_ids, output_attentions=True)
            lines_with_score, n_lines = extract_line_attention(attentions, all_tokens)
            extract_list.append((index, lines_with_score, n_lines))
        index += 1
    return extract_list

# extract_list = explain(model, tokenizer, correct_indices)

# Impact

In [25]:
attack_vector = load_dataset(args.task)

In [26]:
attack_vector_test = attack_vector['test']

In [27]:
attack_vector_test

Dataset({
    features: ['CVE ID', 'explain', 'func_before', 'processed_func'],
    num_rows: 1954
})

In [28]:
attack_vector_test_df = attack_vector_test.to_pandas()
attack_vector_test_df

,CVE ID,explain,func_before,processed_func
0,CVE-2015-1342,gain elevated privileges on the system,"static int cg_getattr(const char *path, struct...","static int cg_getattr(const char *path, struct..."
1,CVE-2016-2416,obtain sensitive information from process memo...,void BufferQueueConsumer::dump(String8& result...,void BufferQueueConsumer::dump(String8& result...
2,CVE-2014-3200,cause a denial of service or possibly have oth...,TemplateURLRef::SearchTermsArgs::ContextualSe...,TemplateURLRef::SearchTermsArgs::ContextualSea...
3,CVE-2016-5199,execute arbitrary code on the system or cause ...,PrefService* DataReductionProxySettings::GetOr...,PrefService* DataReductionProxySettings::GetOr...
4,CVE-2018-6154,overflow a buffer and execute arbitrary code o...,void WebGL2RenderingContextBase::bindVertexAr...,void WebGL2RenderingContextBase::bindVertexArr...
...,...,...,...,...
1949,CVE-2017-0379,discover a secret key,"ecc_decrypt_raw (gcry_sexp_t *r_plain, gcry_se...","ecc_decrypt_raw(gcry_sexp_t *r_plain, gcry_sex..."
1950,CVE-2019-11470,cause a denial-of-service (uncontrolled resour...,static Image *ReadCINImage(const ImageInfo *im...,static Image *ReadCINImage(const ImageInfo *im...
1951,CVE-2012-6545,obtain sensitive information from kernel memory,static int rfcomm_get_dev_list(void __user *ar...,static int rfcomm_get_dev_list(void __user *ar...
1952,CVE-2017-9304,cause a denial of service (stack consumption ),"yydestruct (const char *yymsg, int yytype, YYS...","yydestruct(const char *yymsg, int yytype, YYST..."


In [29]:
result, explain_indices = find_tp(model, tokenizer, args, attack_vector_test_df)

ExtendTextDataset:   0%|          | 0/1954 [00:00<?, ?it/s]

/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [27]:
result, len(explain_indices)

({'test_accuracy': 0.07337526205450734,
  'test_recall': 0.07337526205450734,
  'test_precision': 1.0,
  'test_f1': 0.13671875,
  'test_threshold': 0.5},
 70)

In [30]:
# (sample_idx, lines, n_lines)
extract_list_attack_vector = explain(model, tokenizer, explain_indices, attack_vector_test_df)

ExtendTextDataset:   0%|          | 0/1954 [00:00<?, ?it/s]

  0%|          | 0/1954 [00:00<?, ?it/s]

In [31]:
def extract_top(top_k, extracted_list):
    """
        extracted_list -> [(index, lines, n_lines)], sorted
    """
    result = []
    for i in extracted_list:
        index = i[0]
        lines = i[1]
        n_lines = i[2]
        lines = lines[:max(3, int(top_k*n_lines))]  # get top k        
        lines = '\n'.join([i[1] for i in lines ])  # each item -> (line_idx, content, score)
        result.append((index, lines))
    return result

In [32]:
sub_infos = extract_top(0.1, extract_list_attack_vector)
sub_infos[:3]

[(27,
  'if(data_source_)returndata_source_->DidPassCORSAccessCheck();\nboolWebMediaPlayerImpl::DidPassCORSAccessCheck()const{\nreturnfalse;'),
 (43,
  'ExtensionTtsController*ExtensionTtsController::GetInstance(){\nreturnSingleton<ExtensionTtsController>::get();\n}'),
 (66,
  'inv_txfm_=GET_PARAM(1);\nfwd_txfm_=GET_PARAM(0);\nfwd_txfm_ref=fdct16x16_ref;')]

In [33]:
explain_features = []
for i in sub_infos:
    explain_features.append({
        'func_before': i[1],  # just call func_before to fit input for aspect model
        'explain': attack_vector_test_df.iloc[i[0]]['explain'],
        'orginal_fun': attack_vector_test_df.iloc[i[0]]['func_before']
    })
explain_features_df = pd.DataFrame.from_records(explain_features)

In [34]:
explain_features_dataset = Dataset.from_pandas(explain_features_df)

In [35]:
from dataclasses import dataclass

@dataclass
class AspectArgs:
    model_name = f'tf_board/{args.task}/'
    num_proc = 4
    batch_size = 5
    max_src_length = 1200
    max_des_length = 146
    data_cols = ['func_before', 'explain', 'orginal_fun']
    save_dir = 'tf_board'
    epochs = 100
    grad_acc_steps = 4
    lr = 5e-5
    log_freq = 10
    local_rank = -1
    deepspeed = None
    fp16 = False
    lr_warmup_steps = 200
    weight_decay = 0.05
    task = args.task
    
aspect_args = AspectArgs()

In [36]:
from transformers import AutoTokenizer
codet5p_tokenizer = AutoTokenizer.from_pretrained(aspect_args.model_name)

In [37]:
import torch
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

explain_model = AutoModelForSeq2SeqLM.from_pretrained(aspect_args.model_name).to(device)

In [38]:
def preprocess_function(examples):
    source = [' '.join(ex) for ex in examples["func_before"]]
    target = [' '.join(ex) for ex in examples["explain"]]

    input_feature = codet5p_tokenizer(source, max_length=aspect_args.max_src_length, padding="max_length", truncation=True)
    labels = codet5p_tokenizer(target, max_length=aspect_args.max_des_length, padding="max_length", truncation=True)

    lables = labels["input_ids"].copy()

    return {  "input_ids": input_feature["input_ids"],
              "attention_mask": input_feature["attention_mask"],
              "labels": lables}


tokenized_ds = explain_features_dataset.map(
  preprocess_function,
  remove_columns=aspect_args.data_cols,
  batched=True,
  num_proc=aspect_args.num_proc,
  batch_size=aspect_args.batch_size)

tokenized_ds

Map (num_proc=4):   0%|          | 0/167 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 167
})

In [39]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
  codet5p_tokenizer,
  model=explain_model,
  return_tensors="pt")

In [40]:
sample_dataloader = DataLoader(
  tokenized_ds.with_format("torch"),
  collate_fn=data_collator,
  batch_size=30,
  num_workers=aspect_args.num_proc)

rouge_list = []
gens = []
references = []

for batch in sample_dataloader:
    with torch.no_grad():
        preds = explain_model.generate(
          batch["input_ids"].to(device),
          max_length=aspect_args.max_des_length,
        ).cpu()
        labels = batch["labels"].cpu()
        gens += preds
        references += labels

In [41]:
from commons import clean_generated_str

In [42]:
gens = codet5p_tokenizer.batch_decode(gens, skip_special_tokens=True)
references = codet5p_tokenizer.batch_decode(references, skip_special_tokens=True)

gens = [clean_generated_str(i) for i in gens]
references = [clean_generated_str(i) for i in references]

In [43]:
import evaluate

rouge_metric = evaluate.load("rouge")

results = rouge_metric.compute(predictions=gens, references=references)
results

Using the latest cached version of the module from /data/quy/cache/modules/evaluate_modules/metrics/evaluate-metric--rouge/b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Sun Jul  9 23:52:56 2023) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.


{'rouge1': 0.6485101115245187,
 'rouge2': 0.6103510046043314,
 'rougeL': 0.640399878985465,
 'rougeLsum': 0.6396350726278366}

In [44]:
gens[10]

'the application to crash'

In [45]:
references[10]

'execute arbitrary code on the system'